In [71]:
# Create Machine Learning Model from stock metric data

# Import necessary libraries & modules


import pandas as pd
import csv
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error



In [72]:
# Import .csv file 
# Create DataFrame from .csv file

file = pd.read_csv('/Users/garrettwankel/PycharmProjects/New/Final_df.csv')
df = pd.DataFrame(file)



In [73]:
# Examine DataFrame

df.head()



,,Sym,SP,50-Day MA,200-Day MA,52W Range,MC,P/B,Vol,Avg Vol (3m),Avg Vol (10-day),Sh Out,Float,Insiders,Institutions
0,08-07-2020,crbp,6.61,7.35,6.36,3.29 - 8.78,560.17M,24.43,"1,324,087",1.58M,1.84M,72.49M,71.03M,4.93%,41.18%
1,08-07-2020,ccxi,55.24,56.30,50.83,6.37 - 65.43,3.75B,62.41,"366,281",732.02k,624.86k,67.83M,43.42M,28.93%,67.08%
2,08-07-2020,gwph,109.37,129.58,113.69,67.98 - 172.12,3.81B,5.87,"2,374,243",339.43k,271.39k,31.08M,30.43M,1.12%,83.33%
3,08-07-2020,imvt,27.16,25.17,19.53,8.34 - 29.96,2.11B,22.46,"253,162",427.77k,236.06k,81.81M,25.48M,73.18%,21.10%
4,08-07-2020,alny,143.53,153.62,131.42,73.32 - 167.33,16.55B,14.08,"344,523",769.09k,470.09k,114.82M,113.87M,0.80%,94.74%


In [74]:
# Rename column(s)

df.rename(columns={' ': 'Date'})



,Date,Sym,SP,50-Day MA,200-Day MA,52W Range,MC,P/B,Vol,Avg Vol (3m),Avg Vol (10-day),Sh Out,Float,Insiders,Institutions
0,08-07-2020,crbp,6.61,7.3500,6.3600,3.29 - 8.78,560.17M,24.43,"1,324,087",1.58M,1.84M,72.49M,71.03M,4.93%,41.18%
1,08-07-2020,ccxi,55.24,56.3000,50.8300,6.37 - 65.43,3.75B,62.41,"366,281",732.02k,624.86k,67.83M,43.42M,28.93%,67.08%
2,08-07-2020,gwph,109.37,129.5800,113.6900,67.98 - 172.12,3.81B,5.87,"2,374,243",339.43k,271.39k,31.08M,30.43M,1.12%,83.33%
3,08-07-2020,imvt,27.16,25.1700,19.5300,8.34 - 29.96,2.11B,22.46,"253,162",427.77k,236.06k,81.81M,25.48M,73.18%,21.10%
4,08-07-2020,alny,143.53,153.6200,131.4200,73.32 - 167.33,16.55B,14.08,"344,523",769.09k,470.09k,114.82M,113.87M,0.80%,94.74%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,09-10-2020,imvt,37.13,29.3400,22.2400,8.34 - 38.90,3.15B,11.75,"322,250",535.03k,1.12M,81.81M,30.84M,73.18%,21.10%
78,09-11-2020,crbp,1.88,7.1431,6.5185,1.8100 - 9.7800,199.38M,4.12,"14,201,677",2.82M,11.26M,81.71M,73.07M,4.93%,41.18%
79,09-11-2020,ccxi,48.07,52.6300,51.9200,6.44 - 65.43,3.35B,7.96,"450,650",675.96k,516.13k,68.89M,47.82M,28.93%,67.08%
80,09-11-2020,gwph,102.31,113.2500,111.1800,67.98 - 145.00,3.18B,4.48,"330,252",367.1k,475.57k,31.14M,30.42M,1.12%,83.33%


In [75]:
# Examine datatypes to understand what needs to be converted to numerical values

df.dtypes



                     object
Sym                  object
SP                  float64
50-Day MA           float64
200-Day MA          float64
52W Range            object
MC                   object
P/B                 float64
Vol                  object
Avg Vol (3m)         object
Avg Vol (10-day)     object
Sh Out               object
Float                object
Insiders             object
Institutions         object
dtype: object

In [76]:
# Subset DataFrame for ML model implementation
# Reset index of Subset

df2 = df[df['Sym'] == 'ccxi']
df2.reset_index(inplace=True)



In [77]:
# Remove non-numerical characters from featured columns for uniformity
# Convert object .dtype to float

df2['Vol'] = df2.replace(',', '', regex=True)
df2['Avg Vol (10-day)'] = df2.replace('k', '', regex=True)
df2[['Vol', 'Avg Vol (10-day)']] = df2[['Vol', 'Avg Vol (10-day)']].astype(float)
df2.dtypes



/Users/garrettwankel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/garrettwankel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/garrettwankel/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

index                 int64
                     object
Sym                  object
SP                  float64
50-Day MA           float64
200-Day MA          float64
52W Range            object
MC                   object
P/B                 float64
Vol                 float64
Avg Vol (3m)         object
Avg Vol (10-day)    float64
Sh Out               object
Float                object
Insiders             object
Institutions         object
dtype: object

In [78]:
# Define features variables

X = df2[['50-Day MA', '200-Day MA', 'Vol', 'Avg Vol (10-day)']]
y = df2['SP']



In [79]:
# Split dataset into train & test, specify test_size

train_X, test_x, train_y, test_y = train_test_split(X, y, random_state=2, test_size=.25)



In [80]:
# Create function to determine best 'max_leaf_nodes' value for each model
# Create for loop to iterate through list of possible values
# Print 'max_leaf_nodes' with corresponding 'mean_absolute_error'

def get_mae(max_leaf_nodes, train_X, test_x, train_y, test_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=2)
    model2 = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=10)
    model.fit(train_X, train_y)
    model2.fit(train_X, train_y)
    preds1 = model.predict(test_x)
    preds2 = model2.predict(test_x)
    mae1 = mean_absolute_error(preds1, test_y)
    mae2 = mean_absolute_error(preds2, test_y)
    return(mae1, mae2)

for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, test_x, train_y, test_y)
    print(max_leaf_nodes, my_mae)
    
    

5 (1.1959999999999993, 1.2015980396825512)
50 (0.861999999999999, 1.1405600000000333)
500 (0.861999999999999, 1.1405600000000333)
5000 (0.861999999999999, 1.1405600000000333)


In [81]:
# Create models for predicting
# Train models

model = DecisionTreeRegressor(random_state=2, max_leaf_nodes=50)
model2 = RandomForestRegressor(random_state=2, max_leaf_nodes=50, n_estimators=200, criterion='mae')

model.fit(train_X,train_y)
model2.fit(train_X, train_y).set_params()



RandomForestRegressor(criterion='mae', max_leaf_nodes=50, n_estimators=200,
                      random_state=2)

In [82]:
# Make predictions

preds1 = model.predict(test_x)
preds2 = model2.predict(test_x)



In [83]:
# Score each model to determine it's accuracy based on the returned coeeficient

print(model.score(test_x, test_y))
print(model2.score(test_x, test_y))



0.88515529619445
0.8067276526798364


In [84]:
# Find the 'mean abosolute error' of each model
# ** Helpful to determine how accurate model is **

mae1 = mean_absolute_error(preds1, test_y)
mae2 = mean_absolute_error(preds2, test_y)



In [85]:
# Print 'mean aboslute error'

print(mae1)
print(mae2)



0.861999999999999
1.1268999999999934


In [86]:
# Print True values and corresponding predictions to compare models

test_y, preds1, preds2



(12    53.50
 4     52.20
 18    47.38
 0     55.24
 9     52.81
 Name: SP, dtype: float64,
 array([52.86, 52.99, 48.67, 54.49, 53.65]),
 array([52.3322  , 53.742675, 48.4531  , 54.132725, 53.55365 ]))